In [1]:
import pandas as pd
import numpy as np
import requests
import urllib.request as urllib
import zipfile

https://www.gdeltproject.org/data/documentation/CAMEO.Manual.1.1b3.pdf  
http://data.gdeltproject.org/documentation/GDELT-Event_Codebook-V2.0.pdf  
https://unstats.un.org/unsd/methodology/m49/  
Put in the date range of interest. If separate ranges, add another ` + pd.date_range(...`. Each day will results in 96 separate files (12 hours x 4 15 min intervals).

## Events Table Columns

In [2]:
headers = ['GLOBALEVENTID','SQLDATE','MonthYear','Year','FractionDate','Actor1Code','Actor1Name','Actor1CountryCode','Actor1KnownGroupCode','Actor1EthnicCode','Actor1Religion1Code',
 'Actor1Religion2Code','Actor1Type1Code','Actor1Type2Code','Actor1Type3Code','Actor2Code','Actor2Name','Actor2CountryCode','Actor2KnownGroupCode','Actor2EthnicCode','Actor2Religion1Code',
 'Actor2Religion2Code','Actor2Type1Code','Actor2Type2Code','Actor2Type3Code','IsRootEvent','EventCode','EventBaseCode','EventRootCode','QuadClass','GoldsteinScale','NumMentions',
 'NumSources','NumArticles','AvgTone','Actor1Geo_Type','Actor1Geo_FullName','Actor1Geo_CountryCode','Actor1Geo_ADM1Code','Actor1Geo_ADM2Code','Actor1Geo_Lat','Actor1Geo_Long','Actor1Geo_FeatureID',
 'Actor2Geo_Type','Actor2Geo_FullName','Actor2Geo_CountryCode','Actor2Geo_ADM1Code','Actor2Geo_ADM2Code','Actor2Geo_Lat','Actor2Geo_Long','Actor2Geo_FeatureID','ActionGeo_Type','ActionGeo_FullName',
 'ActionGeo_CountryCode','ActionGeo_ADM1Code','ActionGeo_ADM2Code','ActionGeo_Lat','ActionGeo_Long','ActionGeo_FeatureID','DATEADDED','SOURCEURL']

## Generate URL Datetime Strings

In [3]:
#https://stackoverflow.com/questions/5868130/generating-all-dates-within-a-given-range-in-python
date_strings = [str(x)[:10].replace("-","") for x in \
                pd.date_range('2020-06-01','2020-12-31').tolist()]

In [4]:
time_strings = [str(x) + str(y) for x in ["00","01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23"] \
                 for y in ["0000","1500","3000","4500"]]

In [5]:
datetime_strings = [str(x) + str(y) for x in date_strings for y in time_strings]

In [6]:
events_df = pd.DataFrame(data={},columns=headers)

## Download Data and Populate `events_df`

In [7]:
#iterate thru each date
for i,j in enumerate(datetime_strings):
    #create the URL and get the website data
        #events data
    url = "http://data.gdeltproject.org/gdeltv2/" + j + ".export.CSV.zip"
        #mentions data
    #url = "http://data.gdeltproject.org/gdeltv2/" + i + ".mentions.CSV.zip"
    try:
        #https://stackoverflow.com/questions/6861323/download-and-unzip-file-with-python
        filehandle, _ = urllib.urlretrieve(url)
        zip_file_object = zipfile.ZipFile(filehandle, 'r')
        first_file = zip_file_object.namelist()[0]
        file = zip_file_object.open(first_file)
        content = file.read()

        events = [x.split("\\t") for x in str(content)[2:].split("\\n")]
        temp_df = pd.DataFrame(events,columns=headers)
        temp_df = temp_df.query('ActionGeo_ADM1Code == "USVA"', engine='python')

        events_df = events_df.append(temp_df)
    except:
        continue
    if i % 96 == 0:
        print(j)

20200601000000
20200602000000
20200603000000
20200604000000
20200605000000
20200606000000
20200607000000
20200608000000
20200609000000
20200610000000
20200611000000
20200612000000
20200613000000
20200614000000
20200615000000
20200616000000
20200617000000
20200618000000
20200619000000
20200620000000
20200621000000
20200622000000
20200623000000
20200624000000
20200625000000
20200626000000
20200627000000
20200628000000
20200629000000
20200630000000
20200701000000
20200702000000
20200703000000
20200704000000
20200705000000
20200706000000
20200707000000
20200708000000
20200709000000
20200710000000
20200711000000
20200712000000
20200713000000
20200714000000
20200715000000
20200716000000
20200717000000
20200718000000
20200719000000
20200720000000
20200721000000
20200722000000
20200723000000
20200724000000
20200725000000
20200726000000
20200727000000
20200728000000
20200729000000
20200730000000
20200731000000
20200801000000
20200802000000
20200803000000
20200804000000
20200805000000
2020080600

In [8]:
events_df.shape

(200228, 61)

In [ ]:
#events_df.loc[events_df.ActionGeo_FullName.str.contains("Virginia").fillna("False")]

In [9]:
events_df.head()

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
133,926597211,20200601,202006,2020,2020.4137,,,,,,...,2,"Virginia, United States",US,USVA,,37.768,-78.2057,VA,20200601000000,https://whro.org/news/local-news/10086-photo-g...
189,926597267,20200601,202006,2020,2020.4137,,,,,,...,2,"Virginia, United States",US,USVA,,37.768,-78.2057,VA,20200601000000,https://www.columbian.com/news/2020/may/31/fed...
338,926597416,20200601,202006,2020,2020.4137,,,,,,...,2,"Virginia, United States",US,USVA,,37.768,-78.2057,VA,20200601000000,https://www.940wfaw.com/news/george-floyd-prot...
805,926597883,20200601,202006,2020,2020.4137,CVL,RESIDENTS,,,,...,2,"Virginia, United States",US,USVA,,37.768,-78.2057,VA,20200601000000,https://whro.org/news/local-news/10086-photo-g...
810,926597888,20200601,202006,2020,2020.4137,CVL,RESIDENTS,,,,...,2,"Virginia, United States",US,USVA,,37.768,-78.2057,VA,20200601000000,https://whro.org/news/local-news/10086-photo-g...


Sample code for querying and filtering dataset:

In [16]:
va_events = events_df.query('ActionGeo_FullName.str.contains("Virginia",na=False)', engine='python')

In [10]:
events_df.value_counts("EventCode")

EventCode
020     16582
010     15833
042     14328
051     13873
043     13495
        ...  
106         1
0351        1
0861        1
0344        1
0872        1
Length: 198, dtype: int64

In [11]:
events_df.to_csv("june_to_dec_2020_vaevents.csv")